In [6]:
import os
import sys

In [2]:
%pwd

'c:\\Users\\justi\\Desktop\\MLOps Demo\\MLOPS-Demo-Project\\research'

In [3]:
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\justi\\Desktop\\MLOps Demo\\MLOPS-Demo-Project'

In [ ]:
sys.path.insert(0, '')


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [10]:
from src.mlops_project.constants import *
from src.mlops_project.utils.common import read_yaml, create_directories

ModuleNotFoundError: No module named 'box'

In [ ]:
class ConfigurationManager:
    def __init(self, 
               config_filepath = CONFIG_FILE_PATH, 
               params_filepath = PARAMS_FILE_PATH,
               schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from src.mlops_project import logger
from src.mlops_project.utils.common import get_size

ModuleNotFoundError: No module named 'box'